## Instructions ##

The folder Audio_Files includes all the Exercise 4 audio files.

The folder Models includes the English, Spanish and Italian models you need to solve Exercise 4.

**Important**: You have to install in the notebook the necessary python libraries to solve Exercise 4.

In [15]:
%pip install deepspeech
%pip install jiwer
import subprocess
subprocess.run('pip install --upgrade pip', shell=True)
subprocess.run('pip install soundfile', shell=True)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


CompletedProcess(args='pip install soundfile', returncode=0)

In [9]:
import unicodedata
import re
import pandas as pd
import deepspeech as ds
import numpy as np
import pandas as pd
from pathlib import Path
import jiwer


In [10]:
def normalise_text(text):
    text = "".join(c for c in unicodedata.normalize("NFD", text) if unicodedata.category(c) != "Mn")
    text = re.sub("[^A-Za-z0-9\ ]+", "", text)
    text = re.sub("[\s]+", " ", text)
    return text.lower().strip()

In [11]:
df = pd.read_csv('audio_transcriptions.csv' , header = 0)
df["transcription"] = df.transcription.map(normalise_text)
df

lang                                file  \
0    en          Audio_Files/EN/checkin.wav   
1    en    Audio_Files/EN/checkin_child.wav   
2    en          Audio_Files/EN/parents.wav   
3    en    Audio_Files/EN/parents_child.wav   
4    en         Audio_Files/EN/suitcase.wav   
5    en   Audio_Files/EN/suitcase_child.wav   
6    en        Audio_Files/EN/what_time.wav   
7    en  Audio_Files/EN/what_time_child.wav   
8    en            Audio_Files/EN/where.wav   
9    en      Audio_Files/EN/where_child.wav   
10   en     Audio_Files/EN/my_sentence1.wav   
11   en     Audio_Files/EN/my_sentence2.wav   
12   it       Audio_Files/IT/checkin_it.wav   
13   it       Audio_Files/IT/parents_it.wav   
14   it      Audio_Files/IT/suitcase_it.wav   
15   it     Audio_Files/IT/what_time_it.wav   
16   it         Audio_Files/IT/where_it.wav   
17   es       Audio_Files/ES/checkin_es.wav   
18   es       Audio_Files/ES/parents_es.wav   
19   es      Audio_Files/ES/suitcase_es.wav   
20   es     Audio_Files/ES/what_time_es.wav   
21   es         Audio_Files/ES/where_es.wav   

                                 transcription  
0                    where is the checkin desk  
1                    where is the checkin desk  
2                       i have lost my parents  
3                       i have lost my parents  
4                      i have lost my suitcase  
5                      i have lost my suitcase  
6                        what time is my plane  
7                        what time is my plane  
8          where are the restaurants and shops  
9          where are the restaurants and shops  
10                            how much is this  
11                            where is my food  
12                           dove e il bancone  
13                    ho perso i miei genitori  
14          per favore ho perso la mia valigia  
15                    a che ora e il mio aereo  
16           dove sono i ristoranti e i negozi  
17                 donde estan los mostradores  
18                     he perdido a mis padres  
19              por favor he perdido mi maleta  
20                      a que hora es mi avion  
21  donde estan los restaurantes y las tiendas

**Deepspeech**\
I have made a deepspeech constructor that contains the model and the function to extract the text from the audios. Every line in the csv file will be loaded and find the audio's location by looking at the **file** column.

In [12]:
class DeepSpeech:
    
    def __init__(self, model_name, scorer_name = None):
        model_path = 'Models/' + model_name
        scorer_path = 'Models/' + scorer_name
        
        self.model = ds.Model(model_path)
        if Path(scorer_path).is_file():
            self.model.enableExternalScorer(scorer_path)
        
    def showText(self , sound_arr):
        desired_sr = self.model.sampleRate()
        text = []
        for file in sound_arr:
            audio = read_wave(file)
            audio = (audio.ys*3267).astype(np.int16)
            res = self.model.stt(audio)
            text.append(res)
        return text

**the predicted text from the deepspeech model** \
different model and scorer for different language are fed to the contructor so that it can predict the text not only from English.

In [17]:
import soundfile as sf

class DeepSpeech:
    def __init__(self, model_path, scorer_path):
        self.model = Model(model_path, scorer_path)

    def show_text(self, files):
        text_list = []
        for file in files:
            with sf.SoundFile(file) as sound_file:
                audio = sound_file.read(dtype='int16')
                text = self.model.stt(audio, sound_file.samplerate)
                text_list.append(text)
        return text_list

asr_ds1 = DeepSpeech('deepspeech-0.9.3-models.pbmm', 'deepspeech-0.9.3-models.scorer')
asr_ds2 = DeepSpeech('output_graph_it.pbmm', 'kenlm_it.scorer')
asr_ds3 = DeepSpeech('output_graph_es.pbmm', 'kenlm_es.scorer')

text_predict_en = asr_ds1.show_text(df[df['lang'] == 'en']['file'])
text_predict_it = asr_ds2.show_text(df[df['lang'] == 'it']['file'])
text_predict_es = asr_ds3.show_text(df[df['lang'] == 'es']['file'])
print(text_predict_en)
print(text_predict_it)
print(text_predict_es)

NameError: name 'Model' is not defined

In [18]:
df2 =df.copy()
text_pred = text_predict_en + text_predict_it + text_predict_es
df2['text_pred'] = text_pred
df2

NameError: name 'text_predict_en' is not defined

**evaluator** \
using the **jiwer** library to calculate the error rate of each sentence. Every predicted sentence is used to compare to its correct sentence.

In [19]:
class Evaluator:

    def __init__(self, df: pd.DataFrame):
        self.df = df

    def evaluate(self, asr_name: str = None):
        wers = []
        for y_true, y_pred in zip(self.df.transcription, self.df['text_pred']):
            error = jiwer.wer(y_true, y_pred)
            wer = error*100.
            wers.append(wer)
        self.df['wer'] = wers

evaluate the sentences predicted by the deepspeech model.

In [ ]:
evaluator = Evaluator(df2)
evaluator.evaluate()

In [ ]:
df2

Calculate the average error rate of each language sentences.

In [ ]:
print("wer for english sentences : " , df2[df2['lang'] == 'en']['wer'].mean())
print("wer for italian sentences : " , df2[df2['lang'] == 'it']['wer'].mean())
print("wer for spanish sentences : " , df2[df2['lang'] == 'es']['wer'].mean())

In [ ]:
class DeepSpeech:
    
    def __init__(self, model_name, scorer_name = None):
        model_path = 'Models/' + model_name
        scorer_path = 'Models/' + scorer_name
        
        self.model = ds.Model(model_path)
        if Path(scorer_path).is_file():
            self.model.enableExternalScorer(scorer_path)
        
    def showText(self , sound_arr):
        desired_sr = self.model.sampleRate()
        text = []
        for file in sound_arr:
            language = file.split('/')[1]
            audio = read_wave(file)
            spectrum = audio.make_spectrum()
            if language == 'EN':
                spectrum.high_pass(100)
            else:
                spectrum.high_pass(90)
                
            spectrum.scale(10)
            audio = spectrum.make_wave()
            if language =='IT':
                silence=(0.3 * desired_sr)
                audio_arr = np.append(silence , audio.ys)
            else:
                audio_arr = audio.ys
            audio = (audio_arr*3267).astype(np.int16)
            res = self.model.stt(audio)
            text.append(res)
        return text

In [ ]:
evaluate the new error rate after applying noise reduction.

In [ ]:
asr_ds1 = DeepSpeech('deepspeech-0.9.3-models.pbmm' , 'deepspeech-0.9.3-models.scorer')
asr_ds2 = DeepSpeech('output_graph_it.pbmm' , 'kenlm_it.scorer')
asr_ds3 = DeepSpeech('output_graph_es.pbmm' , 'kenlm_es.scorer')

text_predict_en = asr_ds1.showText(df[df['lang'] == 'en']['file'])
text_predict_it = asr_ds2.showText(df[df['lang'] == 'it']['file'])
text_predict_es = asr_ds3.showText(df[df['lang'] == 'es']['file'])

text_pred = text_predict_en + text_predict_it + text_predict_es
df2['text_pred'] = text_pred

evaluator = Evaluator(df2)
evaluator.evaluate()

print("wer for english sentences : " , df2[df2['lang'] == 'en']['wer'].mean())
print("wer for italian sentences : " , df2[df2['lang'] == 'it']['wer'].mean())
print("wer for spanish sentences : " , df2[df2['lang'] == 'es']['wer'].mean())

In [ ]:
df2

Now the result is getting better.

In [ ]:
pip install pocketsphinx

In [ ]:
from pocketsphinx import Pocketsphinx

**pocketsphinx constructor**

In [ ]:
class pocketsphinx_asr:
    
    def __init__(self):
        self.ps = Pocketsphinx()
        self.text=[]
        
    def loadAudio(self,audioFiles):
        for a in audioFiles:
            self.ps.decode(a)
            res = self.ps.hypothesis()
            self.text.append(res)
    def getText(self):
        return self.text

**load the audios** \
loads the audios and extract the text.

In [ ]:
ps_asr = pocketsphinx_asr()
ps_asr.loadAudio(df['file'])
df3 = df.copy()
df3["text_pred"] = ps_asr.getText()

In [ ]:
**evaluate the sentences**

In [ ]:
evaluator = Evaluator(df3)
evaluator.evaluate()
df3

shows the average error rate of english sentences.

In [ ]:
df3[df3['lang'] == 'en'].wer.mean()

The error rate is very high.

**Vosk** \
extract the text from the audios by using vosk model.

In [ ]:
pip install vosk

In [ ]:
import json
import wave
import vosk
import os

class VoskModel:
    def __init__(self, model_dir):
        self.model = vosk.Model(model_dir)
        self.text = ""
        
    def load_audio(self, file_path):
        with wave.open(file_path) as w:
            rec = vosk.KaldiRecognizer(self.model, w.getframerate())
            data = w.readframes(w.getnframes())
            if rec.AcceptWaveform(data):
                res = json.loads(rec.Result())
                self.text = res["text"]
            else:
                p_text_lst = rec.PartialResult()
                p_str = [json.loads(p)["partial"] for p in p_text_lst]
                self.text = max(p_str, key=len)
    
    def get_text(self):
        return self.text

In [ ]:
model_dir = "Models/vosk_model"
text =[]
for f in df['file']:
    vosk_asr = vosk_model(model_dir)
    vosk_asr.loadAudio(f)
    text.append(vosk_asr.getText())
print(text)

In [ ]:
df4 = df.copy()
df4["text_pred"] = text

In [ ]:
evaluator = Evaluator(df4)
evaluator.evaluate()
df4

In [ ]:
df4[df4['lang'] == 'en'].wer.mean()